<a href="https://colab.research.google.com/github/changhorang/My-Project-1/blob/master/pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

- Table of Contents
 - Basic autograd example 1
 - Basic autograd example 2
 - Loading data from numpy
 - Input pipeline
 - Input pipeline for custom dataset
 - Pretrained model
 - Save and load model

In [2]:
# Basic autograd example

# Create tensors

x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph
y = w*x + b

# Compute gradients
y.backward()

# Print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


In [3]:
# Basic autograd example
# Create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10,3)
y = torch.randn(10,2)

# Build a fully connected layer
linear = nn.Linear(3, 2)
print('w : ', linear.weight)
print('b : ', linear.bias)

# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.01)

# Forward pass
pred = linear(x)

# Compute loss
loss = criterion(pred, y)
print('loss : ', loss.item())

w :  Parameter containing:
tensor([[ 0.5548, -0.4266, -0.1963],
        [ 0.1046, -0.2947, -0.3039]], requires_grad=True)
b :  Parameter containing:
tensor([0.1971, 0.2423], requires_grad=True)
loss :  2.088860511779785


In [4]:
# Backward pass
loss.backward()

# Print out the gradients
print('dL/dw : ', linear.weight.grad)
print('dL/db : ', linear.bias.grad)

# 1-step gradient descent
optimizer.step()

"""
You can also perform gradient descent at the low level.
linear.weight.data.sub_(0.01*linear.weight.grad.data)
linear.bias.data.sub_(0.01*linear.bias.grad.data)
"""

# Print out the loss after 1-step gradient descent
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization : ', loss.item())

dL/dw :  tensor([[ 1.1295, -0.4830, -0.1631],
        [-0.1115, -1.1001, -0.5975]])
dL/db :  tensor([0.4958, 0.6452])
loss after 1 step optimization :  2.0513834953308105


In [5]:
# Loading data from numpy

# Create a numpy array
x = np.array([[1,2], [3,4]])

# Convert the numpy array to a torch tensor
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array
z = y.numpy()

In [6]:
# Input pipeline

# Download and construct CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data',
                                             train = True,
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)


Extracting ../../data/cifar-10-python.tar.gz to ../../data
torch.Size([3, 32, 32])
6


In [10]:
# Data loader (this provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = 64,
                                           shuffle = True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels
images, labels = data_iter.next()

# Actual usage of the data loader is as below
for images, labels in train_loader:
  # Training code should be written here
  pass

In [15]:
# Input pipeline for custom dataset

# You should build your custom dataset as below
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self):
    # TODO
    # 1. Initialize file paths or a list of file names
    pass

  def __getitem__(self, index):
    # TODO
    # 1. Read one data from file (ex : using numpy.fromfile, PIL.Image.open)
    # 2. Preprocess the data (ex : torchvision.Transform)
    # 3. Return a data pair (ex : image and label)
    pass

  def __len__(self):
    # You should change 0 to the total size of your dataset
    return 0

# You can then use the prebuilt data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size = 64)

In [16]:
# Pretrained model

# Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below
for param in resnet.parameters():
  param.requires_grad = False

# Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # 100 is an example

# Forward pass
images = torch.randn(64, 3, 244, 244)
outputs = resnet(images)
print(outputs.size())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([64, 100])


In [17]:
# Save and load the model

# Save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended)
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>